In [38]:
import json
import pandas as pd
from rapidfuzz.distance import LCSseq

In [39]:
input_csv = "../data/raw/normalized_texts_segmented_2.csv"
output_text = "../data/processed/merged_text_final.md"

In [40]:
def find_overlap_end(tail, head):
    length = LCSseq.similarity(tail, head)
    return min(length, len(head))

def merge_chunks(chunks, overlap_size):
    merged_text = chunks[0]
    for i in range(1, len(chunks)):
        prev = chunks[i - 1]
        curr = chunks[i]
        
        k = find_overlap_end(prev[-overlap_size:], curr[:overlap_size])
        merged_text += curr[k:]
    
    return merged_text

In [41]:
data = pd.read_csv(input_csv, encoding="utf-8")
print(data.info())
data = data["processed_text_with_breaks"].tolist()
overlap_size = 250 # 250 characters

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   chunk_id                    270 non-null    int64 
 1   text                        270 non-null    object
 2   processed_text              270 non-null    object
 3   processed_text_with_breaks  270 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.6+ KB
None


In [42]:
merged_text = ""
chunks = []
for c in data:
    try:
        chunks.append(c.strip())
    except Exception as e:
        print(f"Error processing chunk: {e}")
        print(c)
        raise e
print(f"Number of chunks to merge: {len(chunks)}")
merged_text = merge_chunks(chunks, overlap_size)

with open(output_text, "w") as f:
    f.write(merged_text)

Number of chunks to merge: 270


## Sentence Count

In [43]:
import re
delimeters = r'.?!\n'

with open(output_text, "r") as f:
    all_text = f.read()

sentences = re.split(f'[{delimeters}]', all_text)
sentences = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 20]
sentence_count = len(sentences)
print(f"Total number of sentences: {sentence_count}")

print("First 10 sentences:")
for i, sentence in enumerate(sentences[:20]):
    print(f"{i+1}. {sentence}")

Total number of sentences: 13160
First 10 sentences:
1. Tên chính của các chuyên luận là tên Việt Nam, sau tên Việt Nam là tên Latin và những tên Việt Nam thông dụng khác nếu có
2. Đối với dược liệu: Có thể dùng tên quy ước của dược liệu hoặc dùng tên cây, con kèm theo bộ phận dùng làm thuốc để làm tên chuyên luận, những từ chỉ bộ phận dùng làm thuốc để trong dấu ngoặc đơn, ví dụ: (Lá), (Quả), (Thân rễ)
3. Tên quy ước của dược liệu là tên của vị thuốc đã được dùng trong y học cổ truyền, ví dụ: Phù bình, Bạch giới tử
4. Mỗi chuyên luận của Dược điển Việt Nam V (chuyên luận riêng hay phụ lục) là một tiêu chuẩn về chất lượng thuốc hoặc phương pháp kiểm nghiệm thuốc của Việt Nam
5. Nguyên tử lượng các nguyên tố trong Dược điển Việt Nam V là các giá trị đã được thừa nhận ghi trong Phụ lục 18
6. Bảng nguyên tử lượng các nguyên tố
7. Các đơn vị đo lường dùng trong Dược điển Việt Nam V đều tuân theo Luật Đo lường ban hành ngày 11/11/2011 và Nghị định của Chính phủ số 86/2012/NĐ-CP ngày 19/10/2